In [1]:
import matplotlib.pyplot as plt
import numpy as np
import csv
import math
import random

from BSE import market_session

%load_ext autoreload
%autoreload 1
%aimport BSE

In [22]:
# Use this to plot trades of a single experiment
def plot_trades(trial_id):
    prices_fname = trial_id + '_tape.csv'
    x = np.empty(0)
    y = np.empty(0)
    with open(prices_fname, newline='') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            time = float(row[1])
            price = float(row[2])
            x = np.append(x,time)
            y = np.append(y,price)

    plt.plot(x, y, 'x', color='black') 
    
# Use this to run an experiment n times and plot all trades
def n_runs_plot_trades(n, trial_id, start_time, end_time, traders_spec, order_sched):
    x = np.empty(0)
    y = np.empty(0)

    for i in range(n):
        trialId = trial_id + '_' + str(i)
        tdump = open(trialId + '_avg_balance.csv','w')

        market_session(trialId, start_time, end_time, traders_spec, order_sched, tdump, True, False)
        
        tdump.close()

        with open(trialId + '_tape.csv', newline='') as csvfile:
            reader = csv.reader(csvfile)
            for row in reader:
                time = float(row[1])
                price = float(row[2])
                x = np.append(x,time)
                y = np.append(y,price)

    plt.plot(x, y, 'x', color='black');

# !!! Don't use on it's own   
def getorderprice(i, sched, n, mode):
    pmin = min(sched[0][0], sched[0][1])
    pmax = max(sched[0][0], sched[0][1])
    prange = pmax - pmin
    stepsize = prange / (n - 1)
    halfstep = round(stepsize / 2.0)

    if mode == 'fixed':
        orderprice = pmin + int(i * stepsize)
    elif mode == 'jittered':
        orderprice = pmin + int(i * stepsize) + random.randint(-halfstep, halfstep)
    elif mode == 'random':
        if len(sched) > 1:
            # more than one schedule: choose one equiprobably
            s = random.randint(0, len(sched) - 1)
            pmin = min(sched[s][0], sched[s][1])
            pmax = max(sched[s][0], sched[s][1])
        orderprice = random.randint(pmin, pmax)
    return orderprice    

# !!! Don't use on it's own
def make_supply_demand_plot(bids, asks):
    # total volume up to current order
    volS = 0
    volB = 0

    fig, ax = plt.subplots()
    plt.ylabel('Price')
    plt.xlabel('Quantity')
    
    pr = 0
    for b in bids:
        if pr != 0:
            # vertical line
            ax.plot([volB,volB], [pr,b], 'r-')
        # horizontal lines
        line, = ax.plot([volB,volB+1], [b,b], 'r-')
        volB += 1
        pr = b
    if bids:
        line.set_label('Demand')
        
    pr = 0
    for s in asks:
        if pr != 0:
            # vertical line
            ax.plot([volS,volS], [pr,s], 'b-')
        # horizontal lines
        line, = ax.plot([volS,volS+1], [s,s], 'b-')
        volS += 1
        pr = s
    if asks:
        line.set_label('Supply')
        
    if bids or asks:
        plt.legend()
    plt.show()

# Use this to plot supply and demand curves from supply and demand ranges and stepmode
def plot_sup_dem(seller_num, sup_ranges, buyer_num, dem_ranges, stepmode):
    asks = []
    for s in range(seller_num):
        asks.append(getorderprice(s, sup_ranges, seller_num, stepmode))
    asks.sort()
    bids = []
    for b in range(buyer_num):
        bids.append(getorderprice(b, dem_ranges, buyer_num, stepmode))
    bids.sort()
    bids.reverse()
    
    make_supply_demand_plot(bids, asks) 

# plot sorted trades, useful is some situations - won't be used in this worksheet
def in_order_plot(trial_id):
    prices_fname = trial_id + '_tape.csv'
    y = np.empty(0)
    with open(prices_fname, newline='') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            price = float(row[2])
            y = np.append(y,price)
    y = np.sort(y)
    x = list(range(len(y)))

    plt.plot(x, y, 'x', color='black')   

# plot offset function
def plot_offset_fn(offset_fn, total_time_seconds):   
    x = list(range(total_time_seconds))
    offsets = []
    for i in range(total_time_seconds):
        offsets.append(offset_fn(i))
    plt.plot(x, offsets, 'x', color='black')

def plot_profitability(trial_id):
    prices_fname = trial_id + '_strats.csv'
    buyer_pss_list = []
    seller_pps_list = []
    total_pps_list = []
    with open(prices_fname, newline='') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            # print(row)
            buyer_pps = 0
            seller_pps = 0
            for i in range(2, len(row) - 13, 7):
                if row[i + 1][0] == 'B':
                    buyer_pps += float(row[i + 6])
                elif row[i + 1][0] == 'S':
                    seller_pps += float(row[i + 6])
            print(buyer_pps)
            print(seller_pps)
            print(buyer_pps + seller_pps, "\n")
    

In [23]:
# sellers_spec = [('ZIC', 30)]
# buyers_spec = [('ZIC', 30)]

# sellers_spec = [('PRSH', 30, {'k': 4, 's_min': -1, 's_max': 1, 'optimizer': 'PRSH'})]
# buyers_spec = [('PRSH', 30, {'k': 4, 's_min': -1, 's_max': 1, 'optimizer': 'PRSH'})]

sellers_spec = [('PRDE', 30, {'k': 4, 's_min': -1, 's_max': 1, 'F': 0.8, 'optimizer': 'PRDE'})]
buyers_spec = [('PRDE', 30, {'k': 4, 's_min': -1, 's_max': 1, 'F': 0.8, 'optimizer': 'PRDE'})]

traders_spec = {'sellers': sellers_spec, 'buyers': buyers_spec}

sup_range = (60, 60)
dem_range = (140, 140)
# plot_sup_dem(30, [sup_range], 30, [dem_range], 'fixed')

start_time = 0
end_time = 60 * 60 * 24 * 10
supply_schedule = [{'from': start_time, 'to': end_time, 'ranges': [sup_range], 'stepmode': 'fixed'}]
demand_schedule = [{'from': start_time, 'to': end_time, 'ranges': [dem_range], 'stepmode': 'fixed'}]

order_interval = 5
order_sched = {'sup': supply_schedule, 'dem': demand_schedule,
               'interval': order_interval, 'timemode': 'drip-jitter'}

trial_id = 'test'
tdump = open('test_avg_balance.csv','w')
dump_all = True
verbose = True

market_session(trial_id, start_time, end_time, traders_spec, order_sched, tdump, dump_all, verbose)

tdump.close()

plot_trades('test')

plot_profitability('test')

B00: PRDE active_strat=[0]:
[0]: s=-0.861402, start=0.000000, $=0.000000, pps=0.000000
[1]: s=-0.368907, start=0.000000, $=0.000000, pps=0.000000
[2]: s=-0.631135, start=0.000000, $=0.000000, pps=0.000000
[3]: s=-0.951391, start=0.000000, $=0.000000, pps=0.000000


B01: PRDE active_strat=[0]:
[0]: s=+0.049422, start=0.000000, $=0.000000, pps=0.000000
[1]: s=+0.764490, start=0.000000, $=0.000000, pps=0.000000
[2]: s=+0.802204, start=0.000000, $=0.000000, pps=0.000000
[3]: s=+0.778261, start=0.000000, $=0.000000, pps=0.000000


B02: PRDE active_strat=[0]:
[0]: s=+0.443802, start=0.000000, $=0.000000, pps=0.000000
[1]: s=-0.233425, start=0.000000, $=0.000000, pps=0.000000
[2]: s=-0.608985, start=0.000000, $=0.000000, pps=0.000000
[3]: s=-0.923406, start=0.000000, $=0.000000, pps=0.000000


B03: PRDE active_strat=[0]:
[0]: s=+0.130377, start=0.000000, $=0.000000, pps=0.000000
[1]: s=-0.289310, start=0.000000, $=0.000000, pps=0.000000
[2]: s=-0.373924, start=0.000000, $=0.000000, pps=0.0000